In [10]:
import os
import torch
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
import argparse
from utils import *
from gene_expression import *
from pathway_hierarchy import *
import pandas as pd
import yaml
from custom_neural_network import *
from datetime import datetime
import csv
import random
random.seed(0)
np.random.seed(0)


In [11]:
def load_config(config_file):
    with open(config_file, 'r') as file:
        return yaml.safe_load(file)

In [12]:
random.seed(0)
np.random.seed(0)
config = load_config('model_save/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/2024_07_29_13_34_42/config.yml')

In [13]:
config

{'dataset': {'test': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/test.csv',
  'train': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/train.csv',
  'val': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/val.csv',
  'y_test': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/y_test.csv',
  'y_train': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/y_train.csv',
  'y_val': '/12tb_dsk1/danish/Pytorch_Biologically_Informed_Neural_Network/Preprocessed_data/excitory_neurons/Exc_L2-3_CBLN2_LINC02306/y_val.csv'},
 'gene_expression': {'highly_expressed_threshold': 0.95,
  'lowly_expressed_threshold': 0.95,
  'marker': 

In [15]:
import pickle
class TabularDataset(Dataset):
    def __init__(self, count_matrix, label):
        # Read the CSV file
        self.data = count_matrix
        # Separate features and target
        self.features = self.data.values
        self.target = label.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get features and target for a given index
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return features, target

In [16]:
random.seed(0)
np.random.seed(0)
test_datalset_path = config['model_output']['model_save_dir'] + '2024_07_29_13_34_42/test.pkl'
with open(test_dataloader_path, 'rb') as f:
    test_dataset = pickle.load(f)

In [18]:
random.seed(0)
np.random.seed(0)
dataloader_params_path = config['model_output']['model_save_dir'] + '2024_07_29_13_34_42/dataloader_params.pkl'
with open(dataloader_params_path, 'rb') as f:
    dataloader_params = pickle.load(f)

In [26]:
random.seed(0)
np.random.seed(0)
import torch.utils.data as data
test_dataloader = data.DataLoader(test_dataset, **dataloader_params)
for i, j in test_dataloader:
    print(j)
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/anwer/.local/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/anwer/.local/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
TypeError: 'DataLoader' object is not subscriptable


In [20]:
random.seed(0)
np.random.seed(0)
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_list = []
    labels_list = []
    criterion = nn.BCEWithLogitsLoss()
    loss = 0
    with torch.no_grad():  # No need to compute gradients during evaluation
        for features, labels in dataloader:
            outputs = model(features)
            print(outputs)
            #print(outputs)
            predicted = torch.round(torch.sigmoid(outputs.data))
            #print(outputs)
            #print(predicted)
            loss += criterion(outputs, labels)
            #_, predicted = torch.sigmoid(outputs.data)
            predicted_list.append(predicted)
            labels_list.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    #print(total)
    accuracy = 100 * correct / total
    return accuracy, loss, predicted_list, labels_list

In [21]:
random.seed(0)
np.random.seed(0)
models = dict()
models_state_dict = dict()
for i in os.listdir(config['model_output']['model_save_dir']+ '2024_07_29_13_34_42'):
    if 'state' in i:
        continue
        
    elif '.pth' in i:
        models[i] = torch.load(config['model_output']['model_save_dir']+ '2024_07_29_13_34_42'+'/' + i)
        k = i.split('.')[0] + '_state_dict.' + i.split('.')[1]
        models_state_dict[i] = torch.load(config['model_output']['model_save_dir']+ '2024_07_29_13_34_42'+'/' + k)

In [22]:
random.seed(0)
np.random.seed(0)
model = models['best_model_2.pth']
model

CustomNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=582, out_features=843, bias=False)
    (1): Linear(in_features=843, out_features=1, bias=True)
  )
)

In [23]:
random.seed(0)
np.random.seed(0)
accuracy, loss, predicted_list, labels_list = evaluate(models['best_model_2.pth'], test_dataloader)
accuracy

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/anwer/.local/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/anwer/.local/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
TypeError: 'DataLoader' object is not subscriptable


In [25]:
for i, j in test_dataloader:
    print(j)
    break

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/anwer/.local/lib/python3.12/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/anwer/.local/lib/python3.12/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
TypeError: 'DataLoader' object is not subscriptable
